In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score
import glob
import os
import tensorflow as tf
%matplotlib inline

## Model

In [2]:
def conf_mat(pred,truth):
#     print( np.count_nonzero(pred == truth) )
#     print( len(pred) )
    res = [0,0,0,0]
    a = 0
    
    for i in range(len(truth)):
        if truth[i] == 1:
            if truth[i] == pred[i]:
                a = 0
            else:
                a = 2
                
        else:
            if truth[i] == pred[i]:
                a = 1
            else:
                a = 3
                
        res[a] = res[a] + 1
     
    print(res)
    return res

In [9]:
def map_pred(pred):
    res = np.zeros(len(pred))
    for i in range(len(pred)):
        if pred[i] == -1:
            res[i] = 1
            
    return res

## Multi-dataset Model

In [ ]:
input_dir = './../../train/KPI/'

summary = pd.DataFrame(columns=['KPI', 'TP', 'TN', 'FP', 'FN', 'PRECISION', 'RECALL', 'F1_SCORE'])

n_neighbors = 29

for fname in os.listdir(input_dir):
    df  = pd.read_csv(os.path.join(input_dir, fname), index_col='timestamp')
    kpi_name = df['KPI ID'].values[0]
    print(kpi_name)
    df = df.drop(['KPI ID'], axis=1)
    
    # Normalize Values
    normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df = normalized_df.astype({'label': 'int64'})
    
    X = df.value.values.reshape(-1,1)
    

    # Train Model
    model = LocalOutlierFactor(n_neighbors=21)
#     model.fit(X)

    # Make Predictions
    predictions = map_pred(model.fit_predict(X))

    # Compute Confusion Matrix
    cf = conf_mat(predictions,df.label.values) 

    # F1-Score
    prec = 0
    rec = 0
    f1 = 0
    if (cf[0] + cf[2]) != 0:
        prec = cf[0] / (cf[0] + cf[2])
    if (cf[0] + cf[3]) != 0:
        rec  = cf[0] / (cf[0] + cf[3])
    if (prec + rec) != 0:
        f1   = 2 * (prec * rec / (prec+rec))

#     print(f1_score(predictions,y_test))


    summary = summary.append({'KPI': kpi_name, 
                               'TP': cf[0],  
                               'TN': cf[1], 
                               'FP': cf[2], 
                               'FN': cf[3], 
                               'PRECISION': prec, 
                               'RECALL': rec,     
                               'F1_SCORE': f1 }, ignore_index=True)

In [4]:
summary.to_csv('KNN_Res3.csv')

In [8]:
predictions

array([ 1,  1,  1, ...,  1, -1, -1])

In [ ]:
predictions